In [ ]:
%pylab inline
from tmqr.settings import *
from tmqrfeed import DataManager
from pymongo import MongoClient
from tmqrindex.index_base import IndexBase
from datetime import datetime
import re
import pandas as pd


import seaborn as sns
#import cufflinks as cf
#cf.go_offline()

import pandas as pd
import numpy as np
import scipy

#import matplotlib
#%matplotlib notebook

figsize(15,7)

In [ ]:
from sklearn import (pipeline, preprocessing, ensemble, neighbors, linear_model, neural_network, cluster, metrics, decomposition,
                     naive_bayes, calibration, svm, multioutput,
                     feature_selection, discriminant_analysis, model_selection, multiclass
                    )

In [ ]:
client = MongoClient(MONGO_CONNSTR)
db = client[MONGO_DB]

dm = DataManager()

In [ ]:
def get_exo_list(exo_filter='*', return_names=True):
    """
    Return EXO list stored in MongoDB V2
    :param exo_filter: '*' - include all, wildcard is allowed (like, 'ES_Bullish*')
    :param return_names: if True returns names list of EXO, otherwize returns MongoDB data collection list
    :return: list of EXO names
    """
    re_val = exo_filter.replace('*','.*')

    data = db['index_data'].find({'name': re.compile(re_val, re.IGNORECASE)})
    if return_names:
        return [exo['name'] for exo in data]
    else:
        return list(data)

In [ ]:
def universal_fisher_transform(series, transform_with='arctanh'):
    '"transform_with" options - "tanh", "arctanh"'
    
    # Centering the series
    series = series.expanding().apply(lambda x: preprocessing.StandardScaler().fit_transform(x.reshape(-1, 1)
                                                                                                          ).ravel()[-1])

    # limiting it to -0.999 > x < 0.999
    series = series.expanding().apply(lambda x: preprocessing.MinMaxScaler(feature_range=(-0.999,0.999)
                                                                                        ).fit_transform(x.reshape(-1, 1)).ravel()[-1])

    if transform_with == 'arctanh':
        ft_ser = np.arctanh(series)
        
    elif transform_with == 'tanh':
        ft_ser = np.tanh(series)
        
    return ft_ser

In [ ]:
#exo_filter = 'neutralOnly'     # All 
exo_filter = '*'  # ES only
#exo_filter = '*'  # ES Collars (incl Bearish, Bullish, BW and vanilla)

exo_dict = {}
for exo in get_exo_list(exo_filter, return_names=False):
    idx = IndexBase.deserialize(dm, exo, as_readonly=True)
    exo_dict[idx.index_name] = idx

In [ ]:
for exo_name, exo in exo_dict.items():
    exo_df = exo.data
    
    if len(exo_df) < 200:
        print("{0:<70} [NODATA DataLen: {1}]".format(exo.index_name, len(exo_df)))
    elif (datetime.now().date() - exo_df.index.date[-1]).days > 4:
        print("{0:<70} [DELAYED: LastDate: {1}]".format(exo.index_name, exo_df.index[-1]))
    else:
        print("{0:<70} [OK]".format(exo.index_name))

In [ ]:
exo_df = exo_dict['US.6C_ContFutEOD'].data

In [ ]:
vwap = ((exo_df.c * exo_df.v).cumsum() / (exo_df.v).cumsum())

In [ ]:
exo_df.c.plot()
vwap.plot(label='VWAP', legend=True)
exo_df.c.expanding().mean().plot(label='Expanding mean', legend=True);

#### BBands style

In [ ]:
exo_df.c.plot()

#vwap.plot()
#(vwap + 2*vwap.expanding(0).std()).plot()
(vwap - 2*vwap.expanding(0).std()).plot()
#(vwap + 4*vwap.expanding(20).std()).plot()
(vwap - 4*vwap.expanding(0).std()).plot()
'''(vwap + 6*vwap.expanding(20).std()).plot()
(vwap - 6*vwap.expanding(20).std()).plot()
(vwap + 8*vwap.expanding(20).std()).plot()
(vwap - 8*vwap.expanding(20).std()).plot()
(vwap + 10*vwap.expanding(20).std()).plot()
(vwap - 10*vwap.expanding(20).std()).plot()'''

plt.ylim([exo_df.c.min(), exo_df.c.max()])

In [ ]:
close_px = exo_df.c

vwap = ((exo_df.c * exo_df.v).cumsum() / exo_df.v.cumsum())

vwapbands_df = pd.DataFrame()
vwapbands_df['c'] = exo_df.c
#vwapbands_df['v'] = exo_df.v

vwapbands_df['vwap'] = vwap #= ((vwapbands_df['c'] * vwapbands_df['v']).cumsum() / vwapbands_df['v'].cumsum())

bands_mult_maxval = 20# highest value for bands StDev multiplier
bands_mult_step = 2

vwap_exp_std = vwapbands_df['vwap'].expanding().std()
#vwap_exp_std = vwapbands_df['vwap'].rolling(50).std()

for m in np.arange(bands_mult_step, bands_mult_maxval+bands_mult_step, bands_mult_step):
    vwapbands_df['vwap_band_+{}'.format(m)] = vwapbands_df['vwap'] + m * vwap_exp_std
    vwapbands_df['vwap_band_-{}'.format(m)] = vwapbands_df['vwap'] - m * vwap_exp_std

In [ ]:
vwapbands_df.plot(cmap='jet')
plt.ylim([exo_df.c.min(), exo_df.c.max()]);

In [ ]:
bands_vwap_df = vwapbands_df.filter(regex='band')
bands_vwap_diff = bands_vwap_df.sub(vwapbands_df.c, axis=0) * -1

In [ ]:
bands_vwap_diff_morethan0 = bands_vwap_diff[bands_vwap_diff >= 0]
bands_vwap_diff_lessthan0 = bands_vwap_diff[bands_vwap_diff <= 0]

for idx in vwapbands_df.index:
    sup_level = bands_vwap_diff_morethan0.loc[idx].dropna().sort_values()
    res_level = bands_vwap_diff_lessthan0.loc[idx].dropna().sort_values()
    
    if res_level.size != 0:
        vwapbands_df.loc[idx, 'res_level'] = vwapbands_df[res_level.index[-1]].loc[idx]
    
    if sup_level.size != 0:
        vwapbands_df.loc[idx, 'sup_level'] = vwapbands_df[sup_level.index[0]].loc[idx]

In [ ]:
bands_vwap_diff_morethan0 = bands_vwap_diff[bands_vwap_diff >= 0]
bands_vwap_diff_lessthan0 = bands_vwap_diff[bands_vwap_diff <= 0]

vwapbands_df['res_level'] = bands_vwap_df[bands_vwap_diff.isin(bands_vwap_diff_morethan0.min(axis=1))].sum(1)

vwapbands_df['sup_level'] = bands_vwap_df[bands_vwap_diff.isin(bands_vwap_diff_lessthan0.max(axis=1))].sum(1)

In [ ]:
bands_vwap_diff.isin(bands_vwap_diff_lessthan0.max(axis=1))

In [ ]:
vwapbands_df.c.iloc[-500:].plot()

#(vwapbands_df.res_level.shift(1) - vwapbands_df.c.diff().expanding().std()).iloc[-500:].plot(c='r')
vwapbands_df.res_level.shift(1).iloc[-500:].plot(c='r')
vwapbands_df.sup_level.shift(1).iloc[-500:].plot(c='g')

#bands_vwap_df[bands_vwap_diff.isin(bands_vwap_diff_lessthan0.max(axis=1))].sum(1).iloc[-500:].plot()
#bands_vwap_df[bands_vwap_diff.isin(bands_vwap_diff_lessthan0.max(axis=1))].sum(1).iloc[-500:].plot()

In [ ]:
#vwapbands_df.c.expanding().var().plot()
vwapbands_df.c.diff().expanding().std().plot()

In [ ]:
exo_df.c.iloc[-500:].plot()
vwapbands_df.res_level.shift(1).iloc[-500:].plot()
vwapbands_df.sup_level.shift(1).iloc[-500:].plot()

((vwapbands_df.res_level.shift(1) + vwapbands_df.sup_level.shift(1))/2).iloc[-500:].plot()

In [ ]:
vwapbands_df['c'].diff().expanding()

In [ ]:
vwapbands_df[bands_vwap_diff[bands_vwap_diff <= 0].iloc[-1].dropna().sort_values().index[-1]].iloc[-1]

In [ ]:
bands_vwap_diff[bands_vwap_diff <= 0].iloc[-1].dropna().sort_values().index[-1]

In [ ]:
vwapbands_df.vwap - vwapbands_df.c